# Get Azure ML Workspace

In [ ]:
from azureml.core import Workspace, Environment, Experiment
ws = Workspace.from_config()

print('Workspace Name: ' + ws.name, 
      'Azure Region: ' + ws.location, 
      'Resource Group: ' + ws.resource_group, 
      sep = '\n')

# Create Conda Environment with Dependencies

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

predict_env = Environment(name="predict-env")
predict_conda_deps = CondaDependencies.create(pip_packages=['pandas','azureml-core','azureml-dataset-runtime[fuse]','numpy','scikit-learn','azure-kusto-data[pandas]','lightgbm'])
predict_env.python.conda_dependencies = predict_conda_deps

# Create Retraining Pipeline

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.pipeline.core import Pipeline, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration

compute = AmlCompute(ws, "cpu-cluster")

predictRunConfig = RunConfiguration()
predictRunConfig.target = compute
predictRunConfig.environment = predict_env

cluster = "https://iiotmfgdev.westus2.kusto.windows.net"
db = "mfgdb"
query="opcua_raw | where payload contains 'BatchNumber' and unixtime_milliseconds_todatetime(todouble(payload.SourceTimestamp)) between (now(-10m).. now()) | mv-apply payload on (extend key = tostring(bag_keys(payload)[0]) | extend value = payload[key] | summarize b = make_bag(pack(key, value)) ) | evaluate bag_unpack(b)"

modelname_param = PipelineParameter(name="modelname", default_value='iiot-quality-lgbm')
selected_features_param = PipelineParameter(name="selectedfeatures", default_value="S16,S20,S19,S18,S29,S41,S9,S10,S8,S11,S14,S13,S28,S15,S26,S33,S7,S3,S39")
kustocluster_param = PipelineParameter(name="kustocluster", default_value=cluster)
kustodb_param = PipelineParameter(name="kustodb", default_value=db)
kustoquery_param = PipelineParameter(name="kustoquery", default_value=query)
resultdatastorename_param = PipelineParameter(name="resultdatastorename", default_value='iiotmfgdatalakestore')
resultfilename_param = PipelineParameter(name="resultfilename", default_value='tmpresults.csv')

predictStep = PythonScriptStep(name="predict",
    arguments=["--modelname", modelname_param, 
            "--selectedfeatures", selected_features_param, 
            "--kustocluster", kustocluster_param,
            "--kustodb", kustodb_param,
            "--kustoquery",kustoquery_param,
            "--resultdatastorename", resultdatastorename_param,
            "--resultfilename", resultfilename_param],
    script_name="predict.py", 
    runconfig = predictRunConfig, 
    compute_target=compute, 
    source_directory=".",
    allow_reuse=False)

In [ ]:
predictPipeline = Pipeline(workspace=ws, steps=[predictStep])

predictPipeline.validate()

In [ ]:
predictPipelineRun = Experiment(ws, 'Model-Scoring-Pipeline').submit(predictPipeline)

# Publish Retraining Pipeline

In [ ]:
publishedPipeline = predictPipelineRun.publish_pipeline(name="iiot-quality-prediction", description="sample pipeline to perform batch prediction on iiot data",version="0.1")
print("Newly published pipeline id => ",publishedPipeline.id)
print("Newly published pipeline endpoint => ", publishedPipeline.endpoint)